In [62]:
# pip install ipython torch python-dotenv opencv-python matplotlib roboflow ultralytics mss numpy pyautogui pillow transformers easyocr

from IPython import display
import torch
from dotenv import load_dotenv
import shutil
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import roboflow
import ultralytics
from ultralytics import YOLO
import mss
import numpy as np
import time
import pyautogui
from datetime import datetime
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers import AutoTokenizer
import easyocr

In [63]:
display.clear_output()
ultralytics.checks()

Ultralytics 8.3.136  Python-3.11.9 torch-2.7.0+cpu CPU (Intel Core(TM) i7-9700F 3.00GHz)
Setup complete  (8 CPUs, 15.9 GB RAM, 237.6/237.9 GB disk)


In [64]:
# Em apple sillicon, verificar se o MPS está disponível
print(torch.backends.mps.is_available())

False


In [65]:
# Carregar variáveis do ficheiro .env
load_dotenv(override=True)

# Obter variáveis
api_key = os.getenv("ROBOFLOW_API_KEY")
workspace = os.getenv("ROBOFLOW_WORKSPACE")
project_name = os.getenv("ROBOFLOW_PROJECT")
version = int(os.getenv("ROBOFLOW_VERSION", "1"))

In [66]:
ocr = easyocr.Reader(['en'], gpu=False)  # Define idioma e se quer usar GPU

Using CPU. Note: This module is much faster with a GPU.


In [18]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset em Mac tive problemas de permissões e foi necessário dar permissões à pasta de conf. do roboflow:
# - sudo mkdir /Users/davidecarneiro/.config/roboflow (criar pasta onde vai guardar a conf.)
# - sudo chown -R davidecarneiro:staff ~/.config/roboflow (dar permissões ao meu user)
roboflow.login()

rf = roboflow.Roboflow(api_key=api_key) # Faz login no Roboflow com a chave lida

# substituir nome do workspace e do projeto
project = rf.workspace(workspace).project(project_name)
# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(version).download("yolov8")

# Move o dataset para a pasta desejada
shutil.move(dataset.location, "./datasets/candy-crush-saga-v3-1")

# Atualiza o caminho no objeto
dataset.location = "./datasets/candy-crush-saga-v3-1"

# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado
# alterar para:
# train: ./datasets/candy-crush-saga-v3-1/train/images
# val: ./datasets/candy-crush-saga-v3-1/valid/images
# test: ./datasets/candy-crush-saga-v3-1/test/images

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

In [ ]:
# treinar o modelo
# lista de modelos pre-treinados disponível em https://docs.ultralytics.com/models/yolov8/#performance-metrics
model = YOLO("YOLOv8m.pt")  # carregar o modelo pre-treinado que se descarregou

# Treinar o modelo
results = model.train(data='./datasets/candy-crush-saga-v3-1/data.yaml', epochs=100, imgsz=640, device='cpu')  # windows amd cpu and gpu
# results = model.train(data='candy-crush-saga-v3-1', epochs=100, imgsz=640, device=[0, 1]) # intel/windows
# results = model.train(data='candy-crush-saga-v3-1/data.yaml', epochs=100, imgsz=640, device='mps') # apple sillicon

In [3]:
# selecionar a melhor versão do modelo fine-tuned
model = YOLO("runs/detect/train5/weights/best.pt")

In [ ]:
# prever em novas imagens
confidence_level = 0.1
input_path = 'captured_images'
output_path = 'detections'
class_names = model.names

for file in os.listdir(input_path):
    if file.lower().endswith((".png")):
        image = cv2.imread(os.path.join(input_path, file))
        results = model.predict(source=image, conf=confidence_level)  # gerar previsões acima de determinada confiança, e guardar imagens


        output_filename = f"prediction_{file}"
        output_filepath = os.path.join(output_path, output_filename)

        for result in results:
            result.save(filename=output_filepath)
            print("==== Resultados Previsão ====")
            print("Imagem: "+os.path.join(input_path, file))
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
            scores = result.boxes.conf.cpu().numpy()  # Score de confiança
            labels = result.boxes.cls.cpu().numpy()  # Índice da classe

            for i in range(len(boxes)):
                class_id = labels[i]
                class_label = class_names[class_id] if class_id in class_names else "Desconhecido"

                print(f"--- Objeto {i+1} ---")
                print(f"Class: {class_label} (ID: {class_id})")
                print(f"Coordenadas Bounding Box: {boxes[i]}")
                print(f"Confiança: {scores[i]:.4f}")
                print("-------------------")

            print("\n")

# 🎮Testar no jogo, o modelo

###### CONFIGURAÇÕES GERAIS

In [67]:
model = YOLO("runs/detect/train5/weights/best.pt") # Carrega o modelo YOLO treinado
screen_width, screen_height = pyautogui.size() # Tamanho total do ecrã (para eventualmente converter coordenadas para cliques)

# Região do tabuleiro (ajustável via variáveis de ambiente)
BOARD_REGION = {
    "left": int(os.getenv("BOARD_REGION_LEFT", "0")),     # posição X do canto superior esquerdo
    "top": int(os.getenv("BOARD_REGION_TOP", "0")),      # posição Y do canto superior esquerdo
    "width": int(os.getenv("BOARD_REGION_WIDTH", "0")),    # largura da região
    "height": int(os.getenv("BOARD_REGION_HEIGHT", "0"))    # altura da região
}

# Número de linhas e colunas no tabuleiro (ex: 7x7)
ROWS = int(os.getenv("ROWS_GRID", "8"))
COLS = int(os.getenv("COLS_GRID", "8"))

# Região do objetivo (ajustável via variáveis de ambiente)
GOAL_REGION = {
    "left": int(os.getenv("GOAL_REGION_LEFT", "0")),     # posição X do canto superior esquerdo
    "top": int(os.getenv("GOAL_REGION_TOP", "0")),      # posição Y do canto superior esquerdo
    "width": int(os.getenv("GOAL_REGION_WIDTH", "0")),    # largura da região
    "height": int(os.getenv("GOAL_REGION_HEIGHT", "0"))    # altura da região
}

# Garante que as pastas de output existem
os.makedirs("captured_images", exist_ok=True)
os.makedirs("captured_goals", exist_ok=True)
os.makedirs("detections", exist_ok=True)
os.makedirs("detectionsGoal", exist_ok=True)

###### FUNÇÃO DE CAPTURA DE IMAGEM

In [68]:
# função que captura só a região do tabuleiro e devolve a imagem
def capture_board_region():
    # Captura a imagem da região definida como o tabuleiro
    with mss.mss() as sct:
        screenshot = sct.grab(BOARD_REGION)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_height, img_width, _ = img.shape
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

        # Guardar imagem opcionalmente
        img_path = os.path.join('captured_images', f"capture_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height # width, height

###### FUNCAO DE CAPTURA DA REGIAO DOS OBJETIVOS

In [69]:
# função que captura só a região dos objetivos e devolve a imagem
def capture_goal_region():
    # Captura a imagem da região definida como os objetivos
    with mss.mss() as sct:
        screenshot = sct.grab(GOAL_REGION)
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_height, img_width, _ = img.shape
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S-%f")

        # Guardar imagem opcionalmente
        img_path = os.path.join('captured_goals', f"goal_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height # width, height

###### IDENTIFICAR PECAS ADJACENTES E POSSIVEIS COMBINACOES

In [73]:
def sugerir_melhor_jogada(grid):
    global ultima_jogada_objetivo
    ROWS, COLS = len(grid), len(grid[0])
    melhores_trocas = []
    max_combo = 0
    troca_com_color_bomb = None

    os.makedirs("debug_objetivos_crop", exist_ok=True)
    imgGoal, timestampGoal, img_widthGoal, img_heightGoal = capture_goal_region()
    resultsGoal = model.predict(source=imgGoal, conf=0.1)
    caixas = resultsGoal[0].boxes.xyxy.cpu().numpy()
    classes = resultsGoal[0].boxes.cls.cpu().numpy()
    nomes_classes = model.names

    objetivos_detectados = []
    print(f"[DEBUG] Objetivos detetados: {objetivos_detectados}")

    for i, box in enumerate(caixas):
        class_id = int(classes[i])
        nome_doce = nomes_classes[class_id]
        print(f"[DEBUG] Detetado objetivo: {nome_doce} na posição {box.astype(int)}")

        if any(nome_doce in row for row in grid):
            x1, y1, x2, y2 = box.astype(int)
            altura_icon = y2 - y1
            largura_icon = x2 - x1

            # Crop mais robusto: tenta apanhar número abaixo e à direita do ícone
            numero_crop = imgGoal[y2:y2 + int(altura_icon * 1.2), x1:x2 + int(largura_icon * 0.5)]

            nome_ficheiro = f"debug_objetivos_crop/{timestampGoal}_{nome_doce}_{i}.jpg"
            cv2.imwrite(nome_ficheiro, numero_crop)

            resultado_ocr = ocr.readtext(numero_crop)
            numero = None
            if resultado_ocr:
                for linha in resultado_ocr:
                    print(f"[DEBUG OCR] {nome_doce} → {resultado_ocr}")
                    texto = linha[1]
                    print(f"[DEBUG texto OCR]: {texto}")
                    if texto.isdigit():
                        numero = int(texto)
                        break

            if numero is not None:
                objetivos_detectados.append({
                    "cor": nome_doce,
                    "quantidade": numero,
                    "caixa": (x1, y1, x2, y2)
                })
            else:
                print(f"[⚠️ OCR falhou] Não foi possível detetar número para {nome_doce}")
                objetivos_detectados.append({
                    "cor": nome_doce,
                    "quantidade": 1,
                    "caixa": (x1, y1, x2, y2)
                })

    def posicoes_sequencia(g, r, c, cor):
        posicoes = [(r, c)]
        i = c - 1
        while i >= 0 and g[r][i] == cor:
            posicoes.append((r, i))
            i -= 1
        i = c + 1
        while i < COLS and g[r][i] == cor:
            posicoes.append((r, i))
            i += 1
        if len(posicoes) >= 3:
            return posicoes

        posicoes = [(r, c)]
        i = r - 1
        while i >= 0 and g[i][c] == cor:
            posicoes.append((i, c))
            i -= 1
        i = r + 1
        while i < ROWS and g[i][c] == cor:
            posicoes.append((i, c))
            i += 1
        if len(posicoes) >= 3:
            return posicoes
        return []

    def seq_inclui_qualquer_doce_objetivo(posicoes, objetivos):
        for (rr, cc) in posicoes:
            if any(grid[rr][cc] == obj["cor"] for obj in objetivos):
                return True
        return False

    jogadas_objetivos = []
    todas_jogadas = []

    for r in range(ROWS):
        for c in range(COLS):
            if grid[r][c] is None:
                continue
            for dr, dc in [(0, 1), (1, 0)]:
                nr, nc = r + dr, c + dc
                if nr >= ROWS or nc >= COLS or grid[nr][nc] is None:
                    continue

                if grid[r][c] == "color-bomb" or grid[nr][nc] == "color-bomb":
                    troca_com_color_bomb = ((r, c), (nr, nc))

                cor1, cor2 = grid[r][c], grid[nr][nc]
                grid[r][c], grid[nr][nc] = grid[nr][nc], grid[r][c]

                posicoes_r_c = posicoes_sequencia(grid, r, c, cor2) or []
                posicoes_nr_nc = posicoes_sequencia(grid, nr, nc, cor1) or []

                score = max(len(posicoes_r_c), len(posicoes_nr_nc))

                if score >= 3:
                    if score > max_combo:
                        max_combo = score
                        melhores_trocas = [((r, c), (nr, nc))]
                    elif score == max_combo:
                        melhores_trocas.append(((r, c), (nr, nc)))

                    doces_na_seq = [grid[rr][cc] for (rr, cc) in posicoes_r_c + posicoes_nr_nc]
                    objetivos_na_seq = [obj for obj in objetivos_detectados if obj["cor"] in doces_na_seq]
                    if objetivos_na_seq:
                        prioridade = max(obj["quantidade"] for obj in objetivos_na_seq)
                        jogadas_objetivos.append(((r, c), (nr, nc), prioridade, score))
                    else:
                        todas_jogadas.append(((r, c), (nr, nc), 0, score))

                grid[r][c], grid[nr][nc] = grid[nr][nc], grid[r][c]

    if troca_com_color_bomb:
        print(f"\n💡 Jogada com 'color-bomb': {troca_com_color_bomb} (prioritária)")
        return troca_com_color_bomb, 99

    # Priorizar sempre jogadas com objetivos, ordenadas por combo decrescente
    # Verifica se há jogadas com objetivos
    if jogadas_objetivos:
        objetivos_ordenados = sorted(set(j[2] for j in jogadas_objetivos), reverse=True)
        jogada_encontrada = None  # melhor jogada repetida (caso não haja novas)

        for idx, prioridade_atual in enumerate(objetivos_ordenados):
            jogadas_para_prioridade = [j for j in jogadas_objetivos if j[2] == prioridade_atual]
            jogadas_para_prioridade.sort(key=lambda x: x[3], reverse=True)

            print(f"\n🔍 Tentativa com objetivo de prioridade {prioridade_atual}")

            for jogada in jogadas_para_prioridade:
                jogada_coords = jogada[:2]
                if jogada_coords != ultima_jogada_objetivo:
                    ultima_jogada_objetivo = jogada_coords
                    if idx == 0:
                        print(f"\n💡 Jogada de objetivo principal: {jogada_coords} com prioridade {prioridade_atual}")
                    else:
                        print(f"\n💡 Jogada de objetivo secundário: {jogada_coords} com prioridade {prioridade_atual}")
                    return jogada_coords, prioridade_atual

            # Se só houver jogadas repetidas, guardar a melhor (a primeira da lista ordenada)
            if jogada_encontrada is None:
                jogada_encontrada = jogadas_para_prioridade[0]

        # Só chega aqui se nenhuma jogada nova foi encontrada
        if jogada_encontrada:
            print(f"\n🔁 Todas jogadas são repetidas. Reutilizar melhor jogada de objetivo: {jogada_encontrada[:2]} com prioridade {jogada_encontrada[2]}")
            return jogada_encontrada[:2], jogada_encontrada[2]


    # Se não houver jogadas com objetivos, tenta jogadas comuns (maiores combos)
    if todas_jogadas:
        todas_jogadas.sort(key=lambda x: x[3], reverse=True)
        print(f"\n💡 Jogada comum melhor: {todas_jogadas[0][:2]} com combo {todas_jogadas[0][3]}")
        return todas_jogadas[0][:2], todas_jogadas[0][3]

    if melhores_trocas:
        print(f"\n💡 Melhor jogada: {melhores_trocas[0]} que forma grupo de {max_combo}")
        return melhores_trocas[0], max_combo

    print("\n⚠️ Nenhuma jogada possível encontrada.")
    return None, 0


###### LOOP PRINCIPAL

In [74]:
while True:
    # Captura o tabuleiro
    img, timestamp, img_width, img_height = capture_board_region()

    # Faz previsão com YOLO
    results = model.predict(source=img, conf=0.1)
    detections = results[0].boxes.xyxy  # Coordenadas (x1, y1, x2, y2)

    # Define dimensões das células da grelha
    cell_width = img_width // COLS
    cell_height = img_height // ROWS

    grid = [[None for _ in range(COLS)] for _ in range(ROWS)]  # Cria grelha vazia

    if len(detections) > 0:
        print(f"\n✅ Detetou {len(detections)} objetos.")

        # Guarda imagem com bounding boxes
        annotated_frame = results[0].plot()
        result_path = os.path.join('detections', f"result_{timestamp}.jpg")
        cv2.imwrite(result_path, annotated_frame)

        # Processa cada deteção
        for i, (x1, y1, x2, y2) in enumerate(detections.tolist()):
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            col = center_x // cell_width
            row = center_y // cell_height

            class_id = int(results[0].boxes.cls[i].item())
            class_name = model.names[class_id]

            if 0 <= row < ROWS and 0 <= col < COLS:
                grid[row][col] = class_name

            scaled_x = int((center_x / img_width) * screen_width)
            scaled_y = int((center_y / img_height) * screen_height)

        # Imprime o mapa atual do tabuleiro
        print("📌 Mapa do tabuleiro:")
        for row in grid:
            print(row)

        print("\n🔄 Combinações possíveis:")
        melhor_troca, tamanho_grupo = sugerir_melhor_jogada(grid)

        if melhor_troca:
            (r1, c1), (r2, c2) = melhor_troca

            # ✅ Refaz a captura do tabuleiro antes de clicar (para estar atualizado)
            img_ref, timestamp_ref, _, _ = capture_board_region()
            results_ref = model.predict(source=img_ref, conf=0.1)
            detections_ref = results_ref[0].boxes.xyxy

            grid_ref = [[None for _ in range(COLS)] for _ in range(ROWS)]
            for i, (x1, y1, x2, y2) in enumerate(detections_ref.tolist()):
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                col = center_x // cell_width
                row = center_y // cell_height

                class_id = int(results_ref[0].boxes.cls[i].item())
                class_name = model.names[class_id]
                if 0 <= row < ROWS and 0 <= col < COLS:
                    grid_ref[row][col] = class_name

            # (Opcional) mostrar o estado atualizado antes da jogada
            print("📌 Tabuleiro atualizado antes da jogada:")
            for row in grid_ref:
                print(row)

            def cell_to_screen(row, col):
                x = BOARD_REGION["left"] + col * cell_width + cell_width // 2
                y = BOARD_REGION["top"] + row * cell_height + cell_height // 2
                return x, y

            x1, y1 = cell_to_screen(r1, c1)
            x2, y2 = cell_to_screen(r2, c2)

            print(f"🖱️ A mover para ({r1}, {c1}) -> ({r2}, {c2})")
            pyautogui.moveTo(x1, y1, duration=0.3)
            pyautogui.click()
            time.sleep(0.2)
            pyautogui.moveTo(x2, y2, duration=0.3)
            pyautogui.click()
        else:
            print("⚠️ Nenhuma jogada possível para clicar.")

    time.sleep(5)  # Pausa antes da próxima leitura (ajustável)



0: 640x640 10 blues, 14 greens, 10 purples, 10 reds, 12 yellows, 400.6ms
Speed: 4.1ms preprocess, 400.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

✅ Detetou 56 objetos.
📌 Mapa do tabuleiro:
[None, 'purple', 'red', 'blue', 'green', 'blue', 'yellow', None]
['red', 'yellow', 'red', 'green', 'purple', 'yellow', 'blue', 'green']
[None, 'blue', 'purple', 'yellow', 'red', 'yellow', 'yellow', None]
['green', 'red', 'green', 'green', 'purple', 'green', 'purple', 'green']
['green', 'yellow', 'red', 'blue', 'green', 'purple', 'purple', 'green']
[None, 'yellow', 'purple', 'green', 'blue', 'yellow', 'blue', None]
['red', 'red', 'green', 'yellow', 'purple', 'yellow', 'yellow', 'red']
[None, 'blue', 'green', 'blue', 'purple', 'red', 'blue', None]

🔄 Combinações possíveis:

0: 640x640 1 blue, 1 green, 351.1ms
Speed: 3.9ms preprocess, 351.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
[DEBUG] Objetivos detetados: []
[DEBUG] Detetado objetivo: green na pos

KeyboardInterrupt: 